# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize 
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
#tf.set_random_seed(30)
tf.random.set_seed(30)

print(tf.__version__)

2.8.0


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('C:/Users/dpdha/Downloads/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('C:/Users/dpdha/Downloads/Project_data/val.csv').readlines())
print(train_doc.shape, val_doc.shape)
batch_size = 40  #experiment with the batch size
img_height=80
imh_width=80


(663,) (100,)


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [ 5, 7, 10, 12, 15, 17, 20, 22, 25, 27 ]  #create a list of image numbers you want to use for a particular video
    x= len(img_idx) # number of image for each video
    y= img_height  # (y,z) is the final size of the input images 
    z= imh_width
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len( source_path) // batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    image_resize = resize(image, (img_height, imh_width, 3))  
                    # resizing for imh_height imgçwidth and 3 channels for colour
                    
                    batch_data[folder,idx,:,:,0] = (image_resize[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image_resize[:,:,1])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image_resize[:,:,2])/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        remaining_data = len( source_path) % batch_size 
        batch = num_batches  # which is batch+1 and batch starts from 0
        batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
        for folder in range(remaining_data): # iterate over the remaining_data batch_size
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                #crop the images and resize them. Note that the images are of 2 different shape 
                #and the conv3D will throw error if the inputs in a batch have different shapes

                image_resize = resize(image, (img_height, imh_width, 3))  
                # resizing for imh_height imgçwidth and 3 channels for colour

                batch_data[folder,idx,:,:,0] = (image_resize[:,:,0])/255 #normalise and feed in the image
                batch_data[folder,idx,:,:,1] = (image_resize[:,:,1])/255 #normalise and feed in the image
                batch_data[folder,idx,:,:,2] = (image_resize[:,:,2])/255 #normalise and feed in the image

            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
        yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

         


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = 'C:/Users/dpdha/Downloads/Project_data/train' # '/notebooks/storage/Final_data/Collated_training/train'
val_path = 'C:/Users/dpdha/Downloads/Project_data/val' # '/notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
# model_conv3d_1 = Sequential()

# input_shape=(num_frames, image_height, image_width, channels)
inp_shape=(10, img_height, imh_width, 3)

print(keras.__version__)

2.8.0


In [7]:
model_conv3d_1 = Sequential()
model_conv3d_1.add(Conv3D(16, (3, 3, 3), padding='same', input_shape=inp_shape ) )
model_conv3d_1.add(Activation('relu'))
model_conv3d_1.add(BatchNormalization())
model_conv3d_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_conv3d_1.add(Conv3D(32, (3, 3, 3), padding='same'))
model_conv3d_1.add(Activation('relu'))
model_conv3d_1.add(BatchNormalization())
model_conv3d_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_conv3d_1.add(Conv3D(64, (3, 3, 3), padding='same'))
model_conv3d_1.add(Activation('relu'))
model_conv3d_1.add(BatchNormalization())
# model_conv3d_1.add(MaxPooling3D(pool_size=(4,4,4)))

model_conv3d_1.add(Flatten())
model_conv3d_1.add(Dense(128, activation='relu'))
model_conv3d_1.add(BatchNormalization())
model_conv3d_1.add(Dropout(0.3))

model_conv3d_1.add(Dense(64, activation='relu'))
model_conv3d_1.add(BatchNormalization())
model_conv3d_1.add(Dropout(0.3))
model_conv3d_1.add(Dense(5, activation='softmax'))

# set the model based on Conv3D
model = model_conv3d_1

### Model based on CNN+RNN 
We shall use Resnet50 as base model in building CNN+RNN


In [8]:
from tensorflow.keras.applications.resnet50 import ResNet50
#  inp_shape=(5, img_height, imh_width, 3)
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(80,80,3))

In [9]:
for layer in resnet.layers[:-3]:
    layer.trainable=False

In [10]:
cnn =Sequential([resnet])
cnn.add(Conv2D(64,(2,2), padding='same'))
# cnn.add(Conv2D(16,(3,3),padding='same'))
cnn.add(Flatten())

In [11]:
print (cnn.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 3, 3, 2048)        23587712  
                                                                 
 conv2d (Conv2D)             (None, 3, 3, 64)          524352    
                                                                 
 flatten_1 (Flatten)         (None, 576)               0         
                                                                 
Total params: 24,112,064
Trainable params: 528,448
Non-trainable params: 23,583,616
_________________________________________________________________
None


In [53]:
model_cnn_rnn = Sequential()
model_cnn_rnn.add(TimeDistributed(cnn,input_shape=(10, 80, 80, 3)))
model_cnn_rnn.add(GRU(16,input_shape=(None, 10, 256),return_sequences=True))
# model_cnn_rnn.add(GRU(8))
model_cnn_rnn.add(Flatten())
model_cnn_rnn.add(Dense(5,activation='softmax'))

In [54]:
print (model_cnn_rnn.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_7 (TimeDis  (None, 10, 576)          24112064  
 tributed)                                                       
                                                                 
 gru_8 (GRU)                 (None, 10, 16)            28512     
                                                                 
 flatten_2 (Flatten)         (None, 160)               0         
                                                                 
 dense_10 (Dense)            (None, 5)                 805       
                                                                 
Total params: 24,141,381
Trainable params: 557,765
Non-trainable params: 23,583,616
_________________________________________________________________
None


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [55]:
# now set the model based on CNN+RNN
model = model_cnn_rnn

In [56]:
optimiser = 'adam' #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy',  # loss=tf.losses.SparseCategoricalCrossentropy(),
              metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_7 (TimeDis  (None, 10, 576)          24112064  
 tributed)                                                       
                                                                 
 gru_8 (GRU)                 (None, 10, 16)            28512     
                                                                 
 flatten_2 (Flatten)         (None, 160)               0         
                                                                 
 dense_10 (Dense)            (None, 5)                 805       
                                                                 
Total params: 24,141,381
Trainable params: 557,765
Non-trainable params: 23,583,616
_________________________________________________________________
None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [57]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [58]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                       patience=5, min_lr=0.0005)   # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [59]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
# model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                    callbacks=callbacks_list, validation_data=val_generator, 
#                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:/Users/dpdha/Downloads/Project_data/train ; batch size = 40
Epoch 1/50
17/17 [==============================] - ETA: 0s - loss: 0.9822 - categorical_accuracy: 0.2044Source path =  C:/Users/dpdha/Downloads/Project_data/val ; batch size = 40

Epoch 1: saving model to model_init_2022-05-1817_25_15.109741\model-00001-0.98222-0.20441-1.03254-0.24167.h5
17/17 [==============================] - 128s 7s/step - loss: 0.9822 - categorical_accuracy: 0.2044 - val_loss: 1.0325 - val_categorical_accuracy: 0.2417 - lr: 0.0010
Epoch 2/50
17/17 [==============================] - ETA: 0s - loss: 0.8233 - categorical_accuracy: 0.1353
Epoch 2: saving model to model_init_2022-05-1817_25_15.109741\model-00002-0.82326-0.13529-1.03579-0.15000.h5
17/17 [==============================] - 137s 8s/step - loss: 0.8233 - categorical_accuracy: 0.1353 - val_loss: 1.0358 - val_categorical_accuracy: 0.1500 - lr: 0.0010
Epoch 3/50
17/17 [==============================] - ETA: 0s - loss: 0.8833 - categor

In [ ]:
print('ONS OCVN')